# AWS micro ETL sample

## Import required libraries

In [1]:
import pandas as pd
import requests
import io
import matplotlib.pyplot as plt
import boto3
import datetime

## Read file

In [2]:
s3=boto3.client(
    's3',
    aws_access_key_id='*******',
    aws_secret_access_key='*********'
)

In [3]:
obj = s3.get_object(
    Bucket='aspen-capital-raw-data',
Key='data_engineer_raw_data.xlsx')

In [4]:
data = obj['Body'].read()
# print(data)
borrower = pd.read_excel(data, engine='openpyxl', sheet_name='borrower')
display(borrower.head())

,id,full_name,street,city,state,zip_code,phone_home,phone_cell,email
0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,Kimball Bruyns,NaN,Waterbury,Connecticut,6705,NaN,NaN,NaN
1,0c53b401-def5-4c5e-9981-fc7ec92ac230,Claudette MacColgan,2 Roth Alley,Memphis,Tennessee,38104,615-897-8189,407-427-8986,cmaccolgan1@amazonaws.com
2,6a91cf5a-6166-4e01-bdda-00953de327a3,Vanny Fanshaw,NaN,Huntington,West Virginia,25721,NaN,757-599-0522,NaN
3,560cea0d-a266-4c40-b7d1-1688db8d86b8,West Kalinsky,NaN,San Bernardino,California,92410,NaN,281-593-3658,NaN
4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,Vivie Filpo,13 Golf View Hill,Baltimore,Maryland,21290,410-622-0404,801-297-6131,vfilpo4@networkadvertising.org


In [5]:
role_profile = pd.read_excel(data, engine='openpyxl', sheet_name='role_profile')
display(role_profile.head())

,borrower_id,role_profile
0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,co-borrower
1,0c53b401-def5-4c5e-9981-fc7ec92ac230,co-borrower
2,6a91cf5a-6166-4e01-bdda-00953de327a3,co-borrower
3,560cea0d-a266-4c40-b7d1-1688db8d86b8,co-borrower
4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,co-borrower


# ETL

### USER_PROFILE

In [6]:
user_profile_df=borrower.copy()
user_profile_df=user_profile_df.rename(columns={"id":"user_profile_id"})
user_profile_df = user_profile_df[user_profile_df['user_profile_id'].notna()]
user_profile_df = user_profile_df[user_profile_df['full_name'].notna()]
user_profile_df[['first_name','last_name']]=user_profile_df.full_name.str.split(n=1,expand=True)
user_profile_df['created_date']=datetime.datetime.now()
user_profile_df['created_by']='Kriti'
user_profile_df.drop(['full_name', 'street', 'city', 'state', 'zip_code', 'phone_cell', 'email'], axis=1, inplace=True)
user_profile_df['updated_date']=datetime.datetime.now()
user_profile_df['updated_by']='Kriti'
user_profile_df.head()

,user_profile_id,phone_home,first_name,last_name,created_date,created_by,updated_date,updated_by
0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,NaN,Kimball,Bruyns,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti
1,0c53b401-def5-4c5e-9981-fc7ec92ac230,615-897-8189,Claudette,MacColgan,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti
2,6a91cf5a-6166-4e01-bdda-00953de327a3,NaN,Vanny,Fanshaw,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti
3,560cea0d-a266-4c40-b7d1-1688db8d86b8,NaN,West,Kalinsky,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti
4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,410-622-0404,Vivie,Filpo,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti


In [7]:
csv_buffer = io.StringIO()
user_profile_df.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')
s3_resource.Object('aspen-capital-clean-data', 'user_profile.csv').put(Body=csv_buffer.getvalue());

### USER

In [8]:
user_df=user_profile_df.copy()
user_df['user_id']=user_df.index
user_df.drop(['first_name', 'last_name'], axis=1, inplace=True)
user_df=user_df.rename(columns={"created_date":"created","updated_date":"updated"})
user_df.head()

,user_profile_id,phone_home,created,created_by,updated,updated_by,user_id
0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,NaN,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti,0
1,0c53b401-def5-4c5e-9981-fc7ec92ac230,615-897-8189,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti,1
2,6a91cf5a-6166-4e01-bdda-00953de327a3,NaN,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti,2
3,560cea0d-a266-4c40-b7d1-1688db8d86b8,NaN,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti,3
4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,410-622-0404,2022-10-23 03:31:21.309833,Kriti,2022-10-23 03:31:21.312833,Kriti,4


In [9]:
csv_buffer = io.StringIO()
user_df.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')
s3_resource.Object('aspen-capital-clean-data', 'user.csv').put(Body=csv_buffer.getvalue());

### ROLE_PROFILE_TYPE



In [10]:
role_profile_type_df=role_profile.copy()
role_profile_type_df=role_profile_type_df.rename(columns={"borrower_id":"role_profile_type_id","role_profile":"type"})
role_profile_type_df['created']=datetime.datetime.now()
role_profile_type_df['created_by']='Kriti'
role_profile_type_df['updated']=datetime.datetime.now()
role_profile_type_df['updated_by']='Kriti'
role_profile_type_df.head()

,role_profile_type_id,type,created,created_by,updated,updated_by
0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,co-borrower,2022-10-23 03:31:21.966723,Kriti,2022-10-23 03:31:21.966723,Kriti
1,0c53b401-def5-4c5e-9981-fc7ec92ac230,co-borrower,2022-10-23 03:31:21.966723,Kriti,2022-10-23 03:31:21.966723,Kriti
2,6a91cf5a-6166-4e01-bdda-00953de327a3,co-borrower,2022-10-23 03:31:21.966723,Kriti,2022-10-23 03:31:21.966723,Kriti
3,560cea0d-a266-4c40-b7d1-1688db8d86b8,co-borrower,2022-10-23 03:31:21.966723,Kriti,2022-10-23 03:31:21.966723,Kriti
4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,co-borrower,2022-10-23 03:31:21.966723,Kriti,2022-10-23 03:31:21.966723,Kriti


In [11]:
csv_buffer = io.StringIO()
role_profile_type_df.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')
s3_resource.Object('aspen-capital-clean-data', 'role-profile_type.csv').put(Body=csv_buffer.getvalue());

### ROLE_PROFILE

In [12]:
role_profile_df=pd.DataFrame(user_df.loc[:,['user_profile_id','user_id']].set_index('user_profile_id')
                            ).join(role_profile_type_df.set_index('role_profile_type_id'), how='outer')
# pd.concat([user_df.loc[:,['user_profile_id','user_id']].set_index('user_profile_id'), role_profile_type_df.set_index('role_profile_type_id')], axis=1).reset_index()
role_profile_df['role_profile_type_id']=role_profile_df.index
role_profile_df = role_profile_df.reset_index()
role_profile_df['role_profile_id']=role_profile_df.index
role_profile_df=role_profile_df.drop(['type','created','created_by','updated','updated_by','index'], axis=1)
role_profile_df['created']=datetime.datetime.now()
role_profile_df['created_by']='Kriti'
role_profile_df['updated']=datetime.datetime.now()
role_profile_df['updated_by']='Kriti'
role_profile_df.head()

,user_id,role_profile_type_id,role_profile_id,created,created_by,updated,updated_by
0,0,a20d3e1c-d1fb-45a8-b89d-0373f7a29d9d,0,2022-10-23 03:31:22.267521,Kriti,2022-10-23 03:31:22.267521,Kriti
1,1,0c53b401-def5-4c5e-9981-fc7ec92ac230,1,2022-10-23 03:31:22.267521,Kriti,2022-10-23 03:31:22.267521,Kriti
2,2,6a91cf5a-6166-4e01-bdda-00953de327a3,2,2022-10-23 03:31:22.267521,Kriti,2022-10-23 03:31:22.267521,Kriti
3,3,560cea0d-a266-4c40-b7d1-1688db8d86b8,3,2022-10-23 03:31:22.267521,Kriti,2022-10-23 03:31:22.267521,Kriti
4,4,1cd44b01-06ce-44dc-ad12-27bf24bc3b2e,4,2022-10-23 03:31:22.267521,Kriti,2022-10-23 03:31:22.267521,Kriti


In [13]:
csv_buffer = io.StringIO()
role_profile_df.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')
s3_resource.Object('aspen-capital-clean-data', 'role_profile.csv').put(Body=csv_buffer.getvalue());

### ADDRESS